## Deliverable 2. Create a Customer Travel Destinations Map.

In [ ]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [ ]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

In [ ]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
lowest_temp = float(input("What is lowest temperature you would accept for your vacation? "))
highest_temp = float(input("What is lowest temperature you would accept for your vacation? "))


In [ ]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
matching_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= highest_temp) &
                                       (city_data_df["Max Temp"] >= lowest_temp)]
print(f'There were {len(matching_cities_df)} cites that met your temperture profile.')
matching_cities_df

In [ ]:
# 4a. Determine if there are any empty rows.
matching_cities_df.isnull().sum()

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
matching_cities_df = matching_cities_df.dropna()

print(f"There were {len(matching_cities_df)} cities that matched your temperature profile.")
matching_cities_df.head()

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = matching_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# # Set the parameters to search for a hotel in Paris.
# params = {
#     "radius": 5000,
#     "types": "lodging",
#     "key": google_api_key,
#     "location": "48.8566, 2.3522"}
# # Use base URL to search for hotels in Paris.
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# # Make request and get the JSON data from the search.
# hotels = requests.get(base_url, params=params).json()

# hotels

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
count_hotels = 0
count_no_hotels = 0

params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        count_hotels += 1

    except (IndexError):
        print("Hotel not found...Skipping City.")
        count_no_hotels += 1
        
# How many hotels were found
print(f"From a total of {count_hotels + count_no_hotels} cities searched there were a total of {count_hotels} hotels found.")
        

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
hotel_df

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Hotels.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [102]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))